In [1]:
# # create empty DataFrame df and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [2]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'
fp_df_model_top_picks = f'df_model_top_picks'

In [3]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
pickle_dump(df, path_data_dump, fp_df_picks)
print(f'len(df): {len(df)}')
print(df)

len(df): 32
   date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12      sym_freq_11      sym_freq_10        sym_freq_9              sym_freq_8                    sym_freq_7                     sym_freq_6                     sym_freq_5                     sym_freq_4                     sym_freq_3 sym_freq_2
0         2023-04-05                 120      [30, 60, 120]          []          []          []          []               []               []           ['SHV']                      []                      ['FTSM']  ['ANET', 'ELF', 'GE', 'SGEN']  ['ACLS', 'AMPH', 'HY', 'ME...  ['AU', 'GBTC', 'LNTH', 'ML...                             []         []
1         2023-04-05                 120  [15, 30, 60, 120]          []          []          []          []               []               []           ['SHV']                      []         ['AU', 'FTSM', 'NVO']  ['ANET', 'ELF', 'GE', 'OR'...  ['ACLS', 'AMPH', 'HY', 'ME...  ['GBTC', 'LNT

In [4]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  my_date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == my_date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if my_date != prev_date:
    if prev_date == '':
      print('\nModel Results:')  
      print(f'{i:<6}{my_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{my_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = my_date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Results:
0     2023-04-05    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : ['SHV']
2                   [30, 60, 120]        sym_freq_8 : []
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : ['SHV']
5                   [15, 30, 60, 120]    sym_freq_8 : []

6     2023-04-04    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : []
8                   [30, 60, 120]        sym_freq_8 : ['NVDA', 'SHV']
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : []
11                  [15, 30, 60, 120]    sym_freq_8 : ['NVDA', 'SHV']

12    2023-04-03    [30, 60,

In [5]:
model_top_picks = [([30, 60, 120], 'sym_freq_9'), ([15, 30, 60, 120], 'sym_freq_12'), ([30, 60, 120], 'sym_freq_8')]

In [6]:
df_model_top_picks = pickle_load(path_data_dump, fp_df_model_top_picks)

z_items   = zip(d_lbk_freq, picks)
for i, item in enumerate(z_items):
  _date, _days_lookback, _sym_freq, _syms = item[0][0], item[0][1], item[0][2], item[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)
  dlb_sym_freq = (_days_lookback, _sym_freq)
  if dlb_sym_freq in model_top_picks and _syms:
    row_add = [_date, _days_lookback, _sym_freq, _syms]
    df_model_top_picks.loc[len(df_model_top_picks)] = row_add
    print(f'added row to df_model_top_picks:\n{row_add}\n')


0 2023-04-05 [30, 60, 120] sym_freq_12 []
1 2023-04-05 [30, 60, 120] sym_freq_9 ['SHV']
added row to df_model_top_picks:
['2023-04-05', [30, 60, 120], 'sym_freq_9', ['SHV']]

2 2023-04-05 [30, 60, 120] sym_freq_8 []
3 2023-04-05 [15, 30, 60, 120] sym_freq_12 []
4 2023-04-05 [15, 30, 60, 120] sym_freq_9 ['SHV']
5 2023-04-05 [15, 30, 60, 120] sym_freq_8 []
6 2023-04-04 [30, 60, 120] sym_freq_12 []
7 2023-04-04 [30, 60, 120] sym_freq_9 []
8 2023-04-04 [30, 60, 120] sym_freq_8 ['NVDA', 'SHV']
added row to df_model_top_picks:
['2023-04-04', [30, 60, 120], 'sym_freq_8', ['NVDA', 'SHV']]

9 2023-04-04 [15, 30, 60, 120] sym_freq_12 []
10 2023-04-04 [15, 30, 60, 120] sym_freq_9 []
11 2023-04-04 [15, 30, 60, 120] sym_freq_8 ['NVDA', 'SHV']
12 2023-04-03 [30, 60, 120] sym_freq_12 []
13 2023-04-03 [30, 60, 120] sym_freq_9 []
14 2023-04-03 [30, 60, 120] sym_freq_8 ['GE', 'NVDA']
added row to df_model_top_picks:
['2023-04-03', [30, 60, 120], 'sym_freq_8', ['GE', 'NVDA']]

15 2023-04-03 [15, 30, 60, 

In [7]:
df_model_top_picks = df_model_top_picks.astype(str)
df_model_top_picks = df_model_top_picks.drop_duplicates()
df_model_top_picks = df_model_top_picks.sort_values(by=['date', 'days_lookback', 'syms_freq'], ascending=False)
df_model_top_picks = df_model_top_picks.reset_index(drop=True)
pickle_dump(df_model_top_picks, path_data_dump, fp_df_model_top_picks)
print(f'len(df_model_top_picks): {len(df_model_top_picks)}')
print(f'{fp_df_model_top_picks}:\n{df_model_top_picks}')

len(df_model_top_picks): 21
df_model_top_picks:
          date  days_lookback   syms_freq          symbols
0   2023-04-05  [30, 60, 120]  sym_freq_9          ['SHV']
1   2023-04-04  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
2   2023-04-03  [30, 60, 120]  sym_freq_8   ['GE', 'NVDA']
3   2023-03-31  [30, 60, 120]  sym_freq_8         ['NVDA']
4   2023-03-30  [30, 60, 120]  sym_freq_9          ['SHV']
5   2023-03-30  [30, 60, 120]  sym_freq_8         ['NVDA']
6   2023-03-29  [30, 60, 120]  sym_freq_9          ['SHV']
7   2023-03-28  [30, 60, 120]  sym_freq_9          ['SHV']
8   2023-03-27  [30, 60, 120]  sym_freq_9          ['SHV']
9   2023-03-27  [30, 60, 120]  sym_freq_8           ['GE']
10  2023-03-24  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
11  2023-03-23  [30, 60, 120]  sym_freq_9          ['SHV']
12  2023-03-22  [30, 60, 120]  sym_freq_9          ['SHV']
13  2023-03-22  [30, 60, 120]  sym_freq_8         ['FTSM']
14  2023-03-21  [30, 60, 120]  sym_freq_9          ['SHV']
15  2023